### **Feature Movies Genres collection BoxOfficeMojo**

In [0]:
# package use
import pandas as pd
import pickle
import glob
import numpy as np

import warnings
warnings.filterwarnings("ignore")
from collections import Counter

import requests
import urllib
import urllib.request
import time
import re
import string
import matplotlib.pyplot as plt
import seaborn as sns
from bs4 import BeautifulSoup
from IPython.core.display import HTML

In [3]:
# read table
Boxofficemojo_genre =  pd.read_csv("Boxofficemojo_genre.csv", encoding="iso-8859-1")
print(len(Boxofficemojo_genre))
Boxofficemojo_genre.head(1)

2400


,year,bomrank,bomtitle,imdblink,bomlink
0,2004,1,Shrek 2,http://www.imdb.com/title/tt0298148/,http://www.boxofficemojo.com/movies/?id=shrek2...


#### **Collect from year 2004 to 2015**

In [0]:
# fillin genres text using bomlink
Boxofficemojo_genre['genre_text'] = np.nan

# extraction function
def bomlink_extraction(url_):
  response = requests.get(url_)
  soup = BeautifulSoup(response.text, "html.parser")
  filter_out = soup.find_all("div", {"class": "a-section a-spacing-none"})
  filter_out_textList = [filter_out[i].text.strip().split(", ")[0] for i in range(len(filter_out))]
  # split and return a list of genres
  split_genre_text_str = np.nan
  for i in range(len(filter_out_textList)):
    if "Genres" in filter_out_textList[i]:
      genre_text_str = filter_out_textList[i]
      split_genre_text_str = genre_text_str.replace("\n", "")[6:].split(" ")
      split_genre_text_str = [str_genre for str_genre in split_genre_text_str if str_genre != ""]
      break
  return split_genre_text_str
# test
print(bomlink_extraction(Boxofficemojo_genre['bomlink'][0]))

# list collection
for i in range(len(Boxofficemojo_genre)):
  Boxofficemojo_genre['genre_text'][i] = bomlink_extraction(Boxofficemojo_genre['bomlink'][i])
  # process
  if i%200 == 0:
    print(i)

In [36]:
Boxofficemojo_genre.head(5)

,year,bomrank,bomtitle,imdblink,bomlink,genre_text
0,2004,1,Shrek 2,http://www.imdb.com/title/tt0298148/,http://www.boxofficemojo.com/movies/?id=shrek2...,"[Adventure, Animation, Comedy, Family, Fantasy..."
1,2004,2,Spider-Man 2,http://www.imdb.com/title/tt0316654/,http://www.boxofficemojo.com/movies/?id=spider...,"[Action, Adventure, Sci-Fi]"
2,2004,3,The Passion of the Christ,http://www.imdb.com/title/tt0335345/,http://www.boxofficemojo.com/movies/?id=passio...,[Drama]
3,2004,4,Meet the Fockers,http://www.imdb.com/title/tt0290002/,http://www.boxofficemojo.com/movies/?id=meetth...,"[Comedy, Romance]"
4,2004,5,The Incredibles,http://www.imdb.com/title/tt0317705/,http://www.boxofficemojo.com/movies/?id=incred...,"[Action, Adventure, Animation, Family]"


In [40]:
# split column of lists into multiple columns 
# https://stackoverflow.com/questions/44663903/pandas-split-column-of-lists-of-unequal-length-into-multiple-columns
genre_df = Boxofficemojo_genre.genre_text.apply(pd.Series).add_prefix('Genre_')
result = pd.concat([Boxofficemojo_genre, genre_df], axis=1, sort=False)
result.head(1)

,year,bomrank,bomtitle,imdblink,bomlink,genre_text,Genre_0,Genre_1,Genre_2,Genre_3,Genre_4,Genre_5,Genre_6,Genre_7
0,2004,1,Shrek 2,http://www.imdb.com/title/tt0298148/,http://www.boxofficemojo.com/movies/?id=shrek2...,"[Adventure, Animation, Comedy, Family, Fantasy...",Adventure,Animation,Comedy,Family,Fantasy,Romance,NaN,NaN


In [0]:
result.to_csv("Boxofficemojo_genre_2004_2015.csv")

#### **Collect from year 2004 to 2015**

In [0]:
# collection Top 200 from website below
bl_2000 = "https://www.boxofficemojo.com/year/2000/?ref_=bo_yl_table_21"
bl_2001 = "https://www.boxofficemojo.com/year/2001/?ref_=bo_yl_table_21" 
bl_2002 = "https://www.boxofficemojo.com/year/2002/?ref_=bo_yl_table_21"
bl_2003 = "https://www.boxofficemojo.com/year/2003/?ref_=bo_yl_table_21"

In [74]:
# create dictionary for movie name and BoxOfficeMojo link
def movie_link_dictionary(url_, numberOfMovies=200):
  result_dic = {}
  response = requests.get(url_)
  soup = BeautifulSoup(response.text, "html.parser")
  # filter and extract 
  table_list = soup.select("a[href*=/?ref_=bo_yld_table_]")
  for i in range(numberOfMovies):
    movie_name = table_list[i].text
    # check if existing name
    if movie_name in result_dic:
      movie_name = movie_name + " other version"
    result_dic[movie_name] = "https://www.boxofficemojo.com" + table_list[i]['href']
  print("dictionary successfully generate!")
  return result_dic

# collection and dataframe formation
def dic_to_frame(url_, year):
  temp_dic = movie_link_dictionary(url_)
  temp_frame = pd.DataFrame(temp_dic.items(), columns=['bomtitle', 'bomlink'])
  temp_frame['year'] = year
  return temp_frame 

Boxofficemojo_genre_2000 = dic_to_frame(bl_2000, 2000)
print(len(Boxofficemojo_genre_2000))
Boxofficemojo_genre_2001 = dic_to_frame(bl_2001, 2001)
print(len(Boxofficemojo_genre_2001))
Boxofficemojo_genre_2002 = dic_to_frame(bl_2002, 2002)
print(len(Boxofficemojo_genre_2002))
Boxofficemojo_genre_2003 = dic_to_frame(bl_2003, 2003)
print(len(Boxofficemojo_genre_2003))

dictionary successfully generate!
200
dictionary successfully generate!
200
dictionary successfully generate!
200
dictionary successfully generate!
200


In [81]:
# Append and merge
Boxofficemojo_genre_2000_2003 = Boxofficemojo_genre_2000.append(Boxofficemojo_genre_2001, ignore_index = True)
Boxofficemojo_genre_2000_2003 = Boxofficemojo_genre_2000_2003.append(Boxofficemojo_genre_2002, ignore_index = True)
Boxofficemojo_genre_2000_2003 = Boxofficemojo_genre_2000_2003.append(Boxofficemojo_genre_2003, ignore_index = True)
print(len(Boxofficemojo_genre_2000_2003))
Boxofficemojo_genre_2000_2003.head(5)

800


,bomtitle,bomlink,year
0,How the Grinch Stole Christmas,https://www.boxofficemojo.com/release/rl305918...,2000
1,Mission: Impossible II,https://www.boxofficemojo.com/release/rl160029...,2000
2,Gladiator,https://www.boxofficemojo.com/release/rl213624...,2000
3,The Perfect Storm,https://www.boxofficemojo.com/release/rl661161...,2000
4,Meet the Parents,https://www.boxofficemojo.com/release/rl677545...,2000


In [83]:
# call bomlink_extraction and fulfill collection
Boxofficemojo_genre_2000_2003['genre_text'] = np.nan
for i in range(len(Boxofficemojo_genre_2000_2003)):
  Boxofficemojo_genre_2000_2003['genre_text'][i] = bomlink_extraction(Boxofficemojo_genre_2000_2003['bomlink'][i])
  # process
  if i%100 == 0:
    print(i)

0
100
200
300
400
500
600
700


In [84]:
Boxofficemojo_genre_2000_2003.head(5)

,bomtitle,bomlink,year,genre_text
0,How the Grinch Stole Christmas,https://www.boxofficemojo.com/release/rl305918...,2000,"[Comedy, Family, Fantasy]"
1,Mission: Impossible II,https://www.boxofficemojo.com/release/rl160029...,2000,"[Action, Adventure, Thriller]"
2,Gladiator,https://www.boxofficemojo.com/release/rl213624...,2000,"[Action, Adventure, Drama]"
3,The Perfect Storm,https://www.boxofficemojo.com/release/rl661161...,2000,"[Action, Adventure, Drama, Thriller]"
4,Meet the Parents,https://www.boxofficemojo.com/release/rl677545...,2000,"[Comedy, Romance]"


In [85]:
# split
genre_df_2000_2003 = Boxofficemojo_genre_2000_2003.genre_text.apply(pd.Series).add_prefix('Genre_')
result_2000_2003 = pd.concat([Boxofficemojo_genre_2000_2003, genre_df_2000_2003], axis=1, sort=False)
result_2000_2003.head(1)

,bomtitle,bomlink,year,genre_text,Genre_0,Genre_1,Genre_2,Genre_3,Genre_4,Genre_5,Genre_6,Genre_7
0,How the Grinch Stole Christmas,https://www.boxofficemojo.com/release/rl305918...,2000,"[Comedy, Family, Fantasy]",Comedy,Family,Fantasy,NaN,NaN,NaN,NaN,NaN


In [0]:
result_2000_2003.to_csv("Boxofficemojo_genre_2000_2003.csv")